In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
import sys
sys.path.append("..") #this is to add the avobe folder to the package directory
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nexus_tool.weap_tools import get_zonal_stats
import fiona
import itertools
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support

In [ ]:
data_folder = os.path.join('Data')
demand_path = os.path.join(data_folder, 'GIS', 'Processed layers', 'Demand_points.gpkg')
cropland_path = os.path.join(data_folder, 'GIS', 'Processed layers', 'cropland_2013_processed_20200227.csv')
output_file = os.path.join(data_folder, 'Cropland and Builtarea', 'cropland.gz')
output_folder = os.path.join(data_folder, 'Cropland and Builtarea')

cropland = gpd.read_file(cropland_path, encoding='utf-8')
cropland.drop(columns=['WKT', 'gen2013', 'area_ha'], inplace=True)
cropland.reset_index(inplace=True)
cropland.rename(columns={'index': 'Demand point', 
                         'prov': 'province', 'wtd_mean': 'wtd'}, inplace=True)
cropland.columns = cropland.columns.str.replace('mean','')
cropland.columns = cropland.columns.str.replace('mea','')
for column in list(cropland)[1:-3]:
    cropland.loc[cropland[column]=='', column] = np.nan

provinces = gpd.read_file(os.path.join('Data', 'GIS', 'Admin', 'Provinces.gpkg'), 
                          encoding='utf-8')
demand_sites = gpd.read_file(demand_path, encoding='utf-8')

In [ ]:
MerchidSudMoroc = 26192
for gdf in [provinces, demand_sites]:
    gdf.to_crs(epsg=MerchidSudMoroc, inplace=True)

for column in list(cropland)[1:-3]:
    cropland[column] = cropland[column].astype('float')
    cropland[column] = cropland[column].interpolate()
cropland['wtd'] = cropland['wtd'].astype('float')

cropland.crs = f'epsg:{MerchidSudMoroc}'
cropland = cropland.loc[cropland.area_m2>=100] #choose

In [ ]:
dff = cropland.loc[cropland.province=='Taroudannt', 'wtd']
cropland.loc[cropland.province=='Taroudannt', 'wtd'] *= 170 * dff.count() / dff.sum()
dff = cropland.loc[cropland.province=='Chtouka-Aït Baha', 'wtd']
cropland.loc[cropland.province=='Chtouka-Aït Baha', 'wtd'] *= 85 * dff.count() / dff.sum()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

provinces.plot(color='white', edgecolor='black', ax=ax)
cropland.plot(column='_srad7', ax=ax, cmap='OrRd', legend=True, cax=cax)
demand_sites.loc[demand_sites['type']=='Catchment'].plot(ax=ax, column='type', legend=True)
for idx, row in demand_sites.loc[demand_sites['type']=='Catchment'].iterrows():
    ax.annotate(text=row['point'], xy=row.geometry.centroid.coords[0],
                 horizontalalignment='center')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

provinces.plot(color='white', edgecolor='black', ax=ax)
cropland.plot(column='wtd', ax=ax, legend=True, cax=cax)

In [ ]:
cropland.loc[cropland.province=='Taroudannt', 'wtd'].max()

In [ ]:
def get_Re(df,Re,mV,D, Ken_visc, axis=1):
    if axis:
        for i in range (1,13):
            _mV = '{}{}'.format(mV, i)
            _Re = '{}{}'.format(Re, i) #already defined in the class properties Re='Re_'
            
            df[_Re] = (df[_mV]*D)/Ken_visc
    else:
        df[Re] = df[mV]*D/Ken_visc
        
    return df

#f=friction coefficient (unitless), k =  Roughness factor (m)
def get_f(df,f, k,D,Re, axis=1):
    if axis:
        for i in range (1,13):
            _Re = '{}{}'.format(Re, i)
            _f = '{}{}'.format(f, i) #already defined in the class properties f='f_'
        
            df[_f] =0.25/(np.log((k/(3.7*D))+(5.74/(df[_Re]**0.9)))**2)
        
    else:
        df[f] = 0.25/(np.log((k/(3.7*D))+(5.74/(df[Re]**0.9)))**2)

    return df

In [ ]:
np.log(float('inf'))

In [ ]:
def create_time_data(data, first_year, end_year):
    df = pd.DataFrame()
    df['Year'] = list(itertools.chain.from_iterable([[year]*12*data.shape[0] for year in range(first_year,end_year+1)]))
    y_number = df.Year.unique().shape[0]
    dff = pd.DataFrame(data.filter(regex='_srad|province|Demand point|area|wtd').melt(id_vars=['Demand point','province','area_m2','wtd']))
    dff.rename(columns={'variable': 'Month', 'value': 'srad'}, inplace=True) 
    dff = dff.join(data.filter(like='_wind').melt())
    dff.rename(columns={'value': 'wind'}, inplace=True)
    dff.drop(columns=['variable'], inplace=True)
    dff['Month'] = dff['Month'].str.replace('_srad','').astype(int)
    dff.sort_values(['province','Month'], inplace=True)
    dff.reset_index(drop=True, inplace=True)
    df = df.join(pd.concat([dff]*y_number, ignore_index=True))
    df['Date'] = pd.to_datetime(df[['Year','Month']].join(pd.DataFrame({'day': [1]*df.shape[0]})))
    return df

In [ ]:
df_cropland = create_time_data(cropland, 2019, 2050)

In [ ]:
df_cropland

In [ ]:
def get_area_share(df, by, value):
    dff = df.groupby(by)[value].sum().copy()
    return df[value] / df.set_index(by).index.map(dff)

In [ ]:
cropland_temp = cropland.copy()
cropland_temp.loc[cropland_temp['province']=='Inezgane-Aït Melloul', 'province'] = 'Taroudannt'
df_cropland['area_share'] = df_cropland['Demand point'].map(get_area_share(cropland_temp, 'province', 'area_m2'))

In [ ]:
os.makedirs(output_folder, exist_ok = True)
df_cropland.to_csv(output_file, index=False)